In [173]:
#importing libraries
import pandas as pd
import tqdm as tqdm

In [183]:
#reading in the data
stats = pd.read_csv("all_stats.csv")

In [175]:
#
stats

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,WS/48,OBPM,DBPM,BPM,VORP,PTS_T,AST_R,STL_R,BLK_R,3P_R
0,A.C. Green,SF,32,PHO,82,36,25.8,2.6,5.4,0.484,...,0.100,-0.4,-0.9,-1.3,0.4,0.889320,0.466739,0.723462,0.697826,0.402634
1,A.C. Green,PF,37,MIA,82,1,17.2,1.8,4.0,0.444,...,0.110,-2.2,0.0,-2.2,-0.1,0.576052,0.274186,0.596349,0.228025,0.000000
2,A.C. Green,PF,33,DAL,83,73,30.0,2.8,5.8,0.483,...,0.093,-1.0,-0.4,-1.4,0.4,0.896924,0.439243,1.136598,0.472416,0.000000
3,A.C. Green,PF,34,DAL,82,68,32.3,3.0,6.5,0.453,...,0.095,-1.1,-0.3,-1.4,0.4,0.918463,0.824362,1.394536,0.708445,0.000000
4,A.C. Green,PF,35,DAL,50,35,18.5,2.2,5.1,0.422,...,0.074,-1.1,-1.1,-2.2,0.0,0.667203,0.302863,0.888889,0.498584,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16028,Željko Rebrača,C,30,DET,30,12,16.3,2.7,4.8,0.552,...,0.133,-2.8,0.2,-2.6,-0.1,0.840864,0.168261,0.297429,1.383621,0.000000
16029,Željko Rebrača,C,29,DET,74,4,15.9,2.6,5.1,0.505,...,0.134,-2.9,0.6,-2.3,-0.1,0.864440,0.271504,0.598029,2.231237,0.000000
16030,Željko Rebrača,C,32,LAC,58,2,16.0,2.3,4.0,0.568,...,0.122,-1.7,0.4,-1.3,0.2,0.717099,0.225462,0.309436,1.693431,0.000000
16031,Željko Rebrača,C,33,LAC,29,2,14.2,1.8,3.3,0.542,...,0.086,-3.7,0.9,-2.8,-0.1,0.588802,0.177611,0.331404,1.764447,0.000000


In [192]:
#fill in missing values with 0
stats = stats.fillna(0)

In [193]:
stats.columns

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year',
       'Pts Won', 'Pts Max', 'Share', 'Team', 'W', 'L', 'W/L%', 'GB', 'PS/G',
       'PA/G', 'SRS', 'Rk', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%',
       'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS',
       'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP', 'PTS_T', 'AST_R', 'STL_R',
       'BLK_R', '3P_R'],
      dtype='object')

In [194]:
#setting up the predictors
predictors = ['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year',
       'W', 'L', 'W/L%', 'GB', 'PS/G',
       'PA/G', 'SRS', 'Rk', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%',
       'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS',
       'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP']

In [195]:
#split data into training and testing sets
train = stats[stats["Year"] < 2025]
test = stats[stats["Year"] == 2025]

In [ ]:
#function to find the average precision
def find_ap(combination):
    actual = combination.sort_values("Share", ascending=False).head(5)
    predicted = combination.sort_values("predictions", ascending=False)
    ps=[]
    found=0
    seen=1
    for index, row in predicted.iterrows():
        if row["Player"] in actual["Player"].values:
            found +=1
            ps.append(found/seen)
        seen +=1
    return sum(ps)/len(ps)



In [199]:
years = list(range(1991, 2026))

In [ ]:
#function to add ranks to the combination dataframe
def add_ranks(combination):
    # Add actual ranks
    combination = combination.sort_values("Share", ascending=False)
    combination["Rk"] = list(range(1, combination.shape[0] + 1))
    # Add predicted ranks
    combination = combination.sort_values("predictions", ascending=False)
    combination["Predicted_Rk"] = list(range(1, combination.shape[0] + 1))
    combination["Diff"] = combination["Rk"] - combination["Predicted_Rk"]
    return combination


In [ ]:
#Backtesting the model
def backtest(stats, model, year, predictors):
    aps = []
    all_predictions = []
    for year in tqdm.tqdm(years[5:]):
        train = stats[stats["Year"] < year]
        test = stats[stats["Year"] == year]

        model.fit(train[predictors], train["Share"])
        predictions = model.predict((test[predictors]))
        predictions = pd.DataFrame(predictions, columns=["predictions"], index=test.index)
        combination = pd.concat([test[["Player", "Share"]], predictions], axis=1)
        combination = add_ranks(combination)
        all_predictions.append(combination)
        aps.append(find_ap(combination))
    return sum(aps)/len(aps), aps, pd.concat(all_predictions)
    

In [ ]:
#adding ratios to the stats dataframe
stat_ratios = stats[["PTS", "AST", "STL", "BLK", "3P", "Year"]].groupby("Year").apply(lambda x: x/x.mean())
stat_ratios = stat_ratios.reset_index(level=0, drop=True)
stats[["PTS_T", "AST_R", "STL_R", "BLK_R", "3P_R"]] = stat_ratios[["PTS", "AST", "STL", "BLK", "3P"]]

C:\Users\Daniel\AppData\Local\Temp\ipykernel_22648\2022889540.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stat_ratios = stats[["PTS", "AST", "STL", "BLK", "3P", "Year"]].groupby("Year").apply(lambda x: x/x.mean())


In [132]:
stats.to_csv("all_stats.csv", index=False)

In [ ]:
#adding stats ratios to the predictors
predictors += ["PTS_T", "AST_R", "STL_R", "BLK_R", "3P_R"]

In [159]:
from sklearn.ensemble import HistGradientBoostingRegressor

hgb = HistGradientBoostingRegressor(random_state=1, max_iter=100)

mean_ap, aps, all_predictions = backtest(stats, hgb, years[5:], predictors)


100%|██████████| 29/29 [00:48<00:00,  1.66s/it]


In [161]:
mean_ap

0.7914464689651651

In [210]:
# See if you're getting better or worse over time
yearly_performance = pd.DataFrame({'Year': years[5:], 'AP': aps})
print(yearly_performance.sort_values('AP').tail(10))

    Year        AP
26  2022  0.862500
24  2020  0.925000
12  2008  0.925000
14  2010  0.925000
19  2015  0.926667
22  2018  0.926667
20  2016  0.942857
28  2024  0.942857
13  2009  0.966667
7   2003  1.000000


In [ ]:
# Assuming you have existing results
yearly_performance = pd.DataFrame({'Year': years[5:], 'AP': aps})

# Filter to recent years
recent_performance = yearly_performance[yearly_performance['Year'] >= 2019]
recent_ap = recent_performance['AP'].mean()

print(f"AP for years >= 2019: {recent_ap:.4f}")
print(f"Number of years: {len(recent_performance)}")

AP for years >= 2010: 0.9313
Number of years: 6


In [ ]:
#importing the models
from sklearn.ensemble import HistGradientBoostingRegressor, RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=1, min_samples_split=5)
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=1)
hgb_model = HistGradientBoostingRegressor(random_state=1, max_iter=100)
#fitting the model
voting_model = VotingRegressor(estimators=[('rf', rf_model), ('gb', gb_model), ('hgb', hgb_model)])
voting_model.fit(train[predictors], train["Share"])
#making predictions
predictions = voting_model.predict(test[predictors])
predictions = pd.DataFrame(predictions, columns=["predictions"], index=test.index)









In [209]:
combination = pd.concat([test[["Player", "Share"]], predictions], axis=1)
ap = find_ap(combination)
add_ranks(combination)





combination.loc[combination["Player"] == "Trae Young"], ap



(           Player  Share  predictions
 14962  Trae Young    0.0     0.155344,
 0.8016666666666667)

In [ ]:

#backtest
mean_ap, aps, all_predictions = backtest(stats, voting_model, years[5:], predictors)

100%|██████████| 29/29 [40:33<00:00, 83.93s/it] 


In [165]:
mean_ap

0.8096465876412737

In [166]:
combination.sort_values("Share", ascending=False).head(10)

,Player,Share,predictions
11645,Nikola Jokić,0.935,0.255345
13672,Shai Gilgeous-Alexander,0.646,0.154796
9945,Luka Dončić,0.572,0.209183
5666,Giannis Antetokounmpo,0.194,0.230050
6626,Jalen Brunson,0.143,0.117840
7285,Jayson Tatum,0.087,0.103313
823,Anthony Edwards,0.018,0.072423
4394,Domantas Sabonis,0.003,0.142944
8939,Kevin Durant,0.001,0.090558
144,Admiral Schofield,0.000,0.004341


In [190]:
add_ranks(combination)

,Player,Share,predictions,Rk,Predicted_Rk,Diff
5675,Giannis Antetokounmpo,0.470,0.241553,3,1,2
11649,Nikola Jokić,0.787,0.234807,2,2,0
13676,Shai Gilgeous-Alexander,0.913,0.196172,1,3,-2
7286,Jayson Tatum,0.311,0.116612,4,4,0
9670,LeBron James,0.016,0.109183,6,5,1
...,...,...,...,...,...,...
15895,Yves Missi,0.000,-0.048733,466,484,-18
8625,Keegan Murray,0.000,-0.053223,276,485,-209
1292,Bilal Coulibaly,0.000,-0.053697,45,486,-441
1963,Bub Carrington,0.000,-0.061753,65,487,-422
